### Iter protocol

#### For loop under the hood

In [5]:
%%HTML
<iframe src='https://gfycat.com/ifr/YearlyWelcomeBlowfish' frameborder='0' scrolling='no' allowfullscreen width='640' height='1185'></iframe>

In [1]:
for value in [1, 2, 3]:
    print(value)

1
2
3


In [2]:
class Counter:
    pass

In [3]:
for value in Counter():
    print(value)

TypeError: 'Counter' object is not iterable

In [ ]:
it = iter([1, 2, 3])
try:
    while True:
        value = next(it)
        print(value)
except StopIteration:
    pass

How to create iterable object? Implement `__iter__` that returns object implementing `__next__`

In [28]:
class Counter:
    def __init__(self, low, high):
        self.current = low
        self.high = high

    def __iter__(self):
        return self

    def __next__(self):
        if self.current > self.high:
            raise StopIteration
        else:
            self.current += 1
            return self.current - 1


for i in Counter(3, 8):
    print(i)

3
4
5
6
7
8


You don't have to return self in `__iter__`

In [7]:
class Counter:
    def __iter__(self):
        return iter([1, 2, 3])

for value in Counter():
    print(value)

1
2
3


Why is this information useful? Infinite or very large sequences. Problem: find first fibonaci sequence number which has a sum of digits greater than 100

In [11]:
from itertools import islice


class FibonacciIterator:
    def __iter__(self):
        self.a = 0
        self.b = 1
        return self

    def __next__(self):
        a, b = self.a, self.b
        self.a = b
        self.b = a + b
        return a


list(islice(FibonacciIterator(), 10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [15]:
from typing import Iterable, Callable, Any

sum_digits = lambda n: sum(map(int, str(n)))

def find_first(iterable: Iterable, predicate: Callable[[Any], bool]):
    for x in iterable:
        if predicate(x):
            return x

find_first(FibonacciIterator(), lambda x: sum_digits(x) > 100)
# next(x for x in FibonaciIterator() if sum_digits(x) > 100)

218922995834555169026

### Generators

Generator is just like a container, but values are generated on the fly as you iterate.

In [16]:
generator = range(10000000)
big_list = list(generator)

from sys import getsizeof

print(getsizeof(generator))
print(getsizeof(big_list))

48
80000056


Generator comprehensions:

In [17]:
%%timeit
power_2 = [i**2 for i in range(10**6)]

292 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
power_2_gen = (i**2 for i in range(10**6))

533 ns ± 13.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [20]:
! pip3.8 install memory_profiler --user

In [21]:
%load_ext memory_profiler

In [22]:
%memit sum([i**2 for i in range(10**6)])

peak memory: 488.84 MiB, increment: 21.51 MiB


In [23]:
%memit sum(i**2 for i in range(10**6))

peak memory: 467.58 MiB, increment: 0.00 MiB


In [26]:
%%timeit
for i in range(10**6):
    if i == 2:
        break

361 ns ± 31.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [27]:
%%timeit
for i in list(range(10**6)):
    if i == 2:
        break

30 ms ± 453 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Yield

In [28]:
def some_generator():
    print("Starting")
    yield 1
    print("Let's come back to where we left off")
    yield 2
    print("Nope. No more yields")

In [29]:
gen = some_generator()

In [30]:
next(gen)

Starting


1

In [31]:
next(gen)

Let's come back to where we left off


2

In [32]:
next(gen)

Nope. No more yields


StopIteration: 

In [33]:
list(some_generator())

Starting
Let's come back to where we left off
Nope. No more yields


[1, 2]

In [34]:
gen = some_generator()

In [35]:
import inspect
inspect.getgeneratorstate(gen)

'GEN_CREATED'

In [36]:
next(gen)
inspect.getgeneratorstate(gen)

Starting


'GEN_SUSPENDED'

In [37]:
list(gen)
inspect.getgeneratorstate(gen)

Let's come back to where we left off
Nope. No more yields


'GEN_CLOSED'

In [38]:
def primitive_range(start: int, stop: int, step: int = 1):
    current = start
    while current < stop:
        yield current
        current += step


for i in primitive_range(0, 4):
    print(i)

0
1
2
3


In [40]:
import inspect
g = primitive_range(0,10)
inspect.getgeneratorlocals(g)

{'start': 0, 'stop': 10, 'step': 1}

In [41]:
next(g)

0

In [43]:
inspect.getgeneratorlocals(g)

{'start': 0, 'stop': 10, 'step': 1, 'current': 0}

In [45]:
next(g)

1

In [47]:
inspect.getgeneratorlocals(g)

{'start': 0, 'stop': 10, 'step': 1, 'current': 1}

How does python know where he stopped in generator - instruction pointer

In [49]:
def simple_gen():
    x = 10
    yield x
    y = "abc"
    yield y

In [50]:
g = simple_gen()
g.gi_frame.f_lasti

-1

In [51]:
next(g)
g.gi_frame.f_lasti

6

In [52]:
import dis
dis.disco(g.gi_code, lasti=g.gi_frame.f_lasti)

  2           0 LOAD_CONST               1 (10)
              2 STORE_FAST               0 (x)

  3           4 LOAD_FAST                0 (x)
    -->       6 YIELD_VALUE
              8 POP_TOP

  4          10 LOAD_CONST               2 ('abc')
             12 STORE_FAST               1 (y)

  5          14 LOAD_FAST                1 (y)
             16 YIELD_VALUE
             18 POP_TOP
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [53]:
import ctypes
import gc
gc.disable()

class A:
    pass


var_address = 0


def f():
    local_var = A()
    global var_address
    var_address = id(local_var)


f()
local_f_var = ctypes.cast(var_address, ctypes.py_object).value
print(f"{local_f_var}")


In [27]:
import ctypes


class A:
    pass


var_address = 0


def f():
    local_var = A()
    global var_address
    var_address = id(local_var)


f()
import gc

gc.collect()
local_f_var = ctypes.cast(var_address, ctypes.py_object).value
print(f"{local_f_var}")

In [54]:
import dis
print(dis.code_info(some_generator))

Name:              some_generator
Filename:          <ipython-input-28-1c1f0d91e5da>
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        2
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE, 0x2000
Constants:
   0: None
   1: 'Starting'
   2: 1
   3: "Let's come back to where we left off"
   4: 2
   5: 'Nope. No more yields'
Names:
   0: print


In [55]:
def infinite_power_2_gen():
    current = 2
    while True:
        yield current
        current *= 2

In [56]:
powers_of_2 = infinite_power_2_gen()
first_4 = powers_of_2[:4]

TypeError: 'generator' object is not subscriptable

In [57]:
first_5_elements = []
for i in range(5):
    first_5_elements.append(next(powers_of_2))
first_5_elements

[2, 4, 8, 16, 32]

In [56]:
first_5_elements = [pair[0] for pair in zip(powers_of_2, range(5))]
first_5_elements

[64, 128, 256, 512, 1024]

In [59]:
from itertools import islice
first_4_powers_gen = islice(infinite_power_2_gen(), 4)
first_4_powers_gen

In [61]:
list(first_4_powers_gen)

[]

In [66]:
def fib_generator():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [67]:
list(islice(fib_generator(), 10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

### Real use case - iterative xml parsing

In [64]:
from pathlib import Path
from typing import Generator
import xml.etree.ElementTree as ET


def iter_tags_from_xml_file(path: Path) -> Generator[ET.Element, None, None]:
    """
    Parses xml file incrementally to not bloat the ram on big xml files
    :param path: path to xml_file
    :return: generator iterating over xml tags
    """
    xml_iterator = iter(ET.iterparse(str(path), events=("start", "end")))
    _, root = next(xml_iterator)
    for event, element in xml_iterator:
        if event == "end":
            yield element
        # without clearing the root element the whole tree is still stored in ram, but created incrementally
        root.clear()

In [65]:
! du -hs ~/faster_xml.xml

20G	/home/gjklv8/faster_xml.xml


In [68]:
tags = iter_tags_from_xml_file(Path("/home/gjklv8/faster_xml.xml"))

In [70]:
%memit keybox_tags = (entry for entry in tags if entry.tag == "Keybox")

peak memory: 468.84 MiB, increment: 0.00 MiB


In [75]:
%memit keybox_elements = sum(1 for x in keybox_tags)

peak memory: 468.92 MiB, increment: 0.00 MiB


### Data pipelines

In [77]:
! pip3.8 install names --user

In [78]:
import names
import random

with open("student_grades.txt", "w") as file:
    for _ in range(25):
        print(f"{names.get_first_name()} {','.join(map(str,random.choices(range(2,6),k=15)))}", file=file)


In [79]:
!cat student_grades.txt

William 3,2,3,5,3,2,5,2,5,3,5,3,3,4,5
Patricia 2,5,5,5,4,4,2,4,3,2,3,5,5,2,3
Gale 2,3,5,4,5,2,3,3,5,3,2,3,5,2,2
Collin 2,2,2,4,2,2,5,5,2,5,4,4,3,5,2
Frank 3,4,2,3,4,2,3,2,4,4,3,4,4,2,2
Donovan 2,3,3,5,5,4,3,3,3,2,2,2,5,5,2
Donna 5,4,4,2,4,5,4,5,3,4,4,4,4,3,2
Melvin 4,4,3,3,5,3,5,4,3,5,2,2,5,5,4
Richard 2,5,5,2,4,4,3,4,3,5,5,2,3,2,3
Erin 3,5,3,4,4,4,3,5,4,2,3,3,2,5,2
Christopher 3,4,5,5,4,5,4,5,2,4,2,5,4,4,3
Ralph 5,5,3,5,4,5,2,2,4,3,3,2,4,3,4
Tom 2,4,2,5,3,3,5,3,2,4,5,2,3,5,4
Brian 4,3,2,5,5,3,5,2,3,3,4,4,4,3,2
Tiffany 2,4,2,2,3,3,3,3,2,2,2,2,4,3,3
Wesley 4,2,5,3,2,2,4,5,5,3,2,5,5,4,2
Andrew 2,2,2,2,3,2,3,2,2,5,2,4,4,3,3
Richard 2,4,5,3,4,2,2,3,3,5,4,5,2,5,5
Preston 2,2,2,3,3,5,4,2,5,5,3,3,5,4,5
Eileen 2,3,3,4,4,5,4,5,4,4,5,2,3,2,3
Peggy 2,2,3,3,4,2,2,4,3,2,3,4,3,4,2
Sarah 2,4,2,4,5,5,4,2,5,4,2,3,4,5,4
Robert 5,5,2,3,2,2,5,5,5,3,3,2,5,4,3
Shannon 5,2,5,2,5,4,3,4,4,5,2,3,5,4,5
Ruben 4,5,2,5,2,4,2,3,5,2,2,4,2,2,3


Find first student that should be expelled
This won't scale:

In [81]:
from pprint import pprint
from statistics import mean


def get_lines(filename: str):
    lines = []
    with open(filename) as file:
        for line in file:
            lines.append(line)
    return lines


def parse_lines(lines: list):
    students_with_grades = []
    for line in lines:
        student, grades_str = line.split()
        grades = [int(grade) for grade in grades_str.split(",")]
        students_with_grades.append((student, grades))
    return students_with_grades


def get_students_with_means(students: list):
    student_with_means = []
    for student, grades in students:
        student_with_means.append((student, mean(grades)))
    return student_with_means


lines = get_lines("student_grades.txt")
students_with_grades = parse_lines(lines)
student_with_means = get_students_with_means(students_with_grades)
student_with_means

[('William', 3.533333333333333),
 ('Patricia', 3.6),
 ('Gale', 3.2666666666666666),
 ('Collin', 3.2666666666666666),
 ('Frank', 3.066666666666667),
 ('Donovan', 3.2666666666666666),
 ('Donna', 3.8),
 ('Melvin', 3.8),
 ('Richard', 3.466666666666667),
 ('Erin', 3.466666666666667),
 ('Christopher', 3.933333333333333),
 ('Ralph', 3.6),
 ('Tom', 3.466666666666667),
 ('Brian', 3.466666666666667),
 ('Tiffany', 2.6666666666666665),
 ('Wesley', 3.533333333333333),
 ('Andrew', 2.7333333333333334),
 ('Richard', 3.6),
 ('Preston', 3.533333333333333),
 ('Eileen', 3.533333333333333),
 ('Peggy', 2.8666666666666667),
 ('Sarah', 3.6666666666666665),
 ('Robert', 3.6),
 ('Shannon', 3.8666666666666667),
 ('Ruben', 3.1333333333333333)]

In [82]:
next(((student, mean) for student, mean in student_with_means if mean < 3.5))

('Gale', 3.2666666666666666)

But this will scale:

In [84]:
from pprint import pprint
from statistics import mean

def get_lines(filename: str):
    lines = []
    with open(filename) as file:
        for line in file:
            lines.append(line)
    return lines


def get_lines_gen(filename: str):
    with open(filename) as file:
        for line in file:
            yield line


def parse_lines_gen(lines):
    for line in lines:
        student, grades_str = line.split()
        grades = [int(grade) for grade in grades_str.split(",")]
        yield (student, grades)
        
def parse_lines(lines):
    students_with_grades = []
    for line in lines:
        student, grades_str = line.split()
        grades = [int(grade) for grade in grades_str.split(",")]
        students_with_grades.append((student, grades))
    return students_with_grades


def get_students_with_means(students):
    student_with_means = []
    for student, grades in students:
        student_with_means.append((student, mean(grades)))
    return student_with_means

def get_students_with_means_gen(students):
    for student, grades in students:
        yield (student, mean(grades))


In [85]:
lines = get_lines_gen("student_grades.txt")
students_with_grades = parse_lines_gen(lines)
student_with_means = get_students_with_means_gen(students_with_grades)

In [46]:
list(student_with_means)

[('Larry', 3.6666666666666665),
 ('Allen', 3.4),
 ('Gertie', 3.7333333333333334),
 ('Arnold', 3.6666666666666665),
 ('Joseph', 3.4),
 ('Grace', 3.2),
 ('Marcia', 3.8),
 ('James', 4.133333333333334),
 ('Colin', 3.4),
 ('Larry', 3.8),
 ('Allan', 3.1333333333333333),
 ('Scott', 3.4),
 ('Michael', 3.2666666666666666),
 ('Lawrence', 3.6666666666666665),
 ('Donna', 3.1333333333333333),
 ('Harley', 3.6),
 ('Thomas', 3.6),
 ('Stuart', 3.4),
 ('William', 3.466666666666667),
 ('Virginia', 3.8),
 ('Carol', 3.4),
 ('Gina', 4),
 ('Amy', 3.8),
 ('Calvin', 3.3333333333333335),
 ('Anthony', 4)]

In [86]:
lines = get_lines_gen("student_grades.txt")
students_with_grades = parse_lines_gen(lines)
student_with_means = get_students_with_means_gen(students_with_grades)
next(((student, mean) for student, mean in student_with_means if mean < 3.5))

('Gale', 3.2666666666666666)

In [87]:
lines = (line for line in open("student_grades.txt"))
splitted_lines = (line.split() for line in lines)
students_with_grades = ((student, [int(grade) for grade in grades_str.split(",")]) for student, grades_str in splitted_lines)
students_with_means = ((student, mean(grades)) for student, grades in students_with_grades)
next(((student, mean) for student, mean in students_with_means if mean < 3.5))

('Gale', 3.2666666666666666)

In [152]:
!ls

cop_materials.ipynb  student_grades.txt  techcrunch.csv
irrelevant.txt	     students.txt	 threading_consumer_producer.py


In [154]:
(line for line in open("fdf"))

FileNotFoundError: [Errno 2] No such file or directory: 'fdf'

In [165]:
(line for line in open("irrelevant.txt") for line in dupa)

<generator object <genexpr> at 0x7fd10fccad60>

In [160]:
(line for line in 1)

TypeError: 'int' object is not iterable

In [163]:
lambdas = []
for i in range(5):
    lambdas.append(lambda: print(dupa))
print(f"{i=}")
for l in lambdas:
    l()

i=4


NameError: name 'dupa' is not defined

### Other generator methods: send, throw, close

In [148]:
def adjustable_counter():
    current = 0
    while True:
        next_val = yield current
        if next_val is not None:
            current = next_val
        current += 1


c = adjustable_counter()

In [149]:
c.send(None)

0

In [150]:
next(c)

1

In [151]:
next(c)

2

In [147]:
c.send(-100) # Spoiler alert - that's how coroutines communitate

-99

In [113]:
next(c)

-98

In [114]:
c.throw(RuntimeError("Sorry"))

RuntimeError: Sorry

In [115]:
c.close()

### Context managers - RAII in Python

In [126]:
with open("irrelevant.txt","w") as file:
    file.write("raii")

Is better than:

In [127]:
file = open("irrelevant.txt","w")
try:
    file.write("raii")
finally:
    file.close()

In [128]:
from threading import Lock

lock = Lock()
x = 10

In [129]:
lock.acquire()
x += 1
lock.release()

In [130]:
with lock:
    x += 1
with lock:
    x += 1

In [135]:
%%HTML
<blockquote class="reddit-card" data-card-created="1570178622"><a href="https://www.reddit.com/r/ProgrammerHumor/comments/bfr1xc/i_love_python_but/">I love Python, but...</a> from <a href="http://www.reddit.com/r/ProgrammerHumor">r/ProgrammerHumor</a></blockquote>
<script async src="//embed.redditmedia.com/widgets/platform.js" charset="UTF-8"></script>

#### How does this work under the hood?

In [133]:
class File:
    def __init__(self, name: str, mode: str = "r"):
        self.name = name
        self.mode = mode
        self.file_handle = None

    def __enter__(self):
        self.file_handle = open(self.name, self.mode)
        return self.file_handle

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("__exit__ called")
        if self.file_handle:
            self.file_handle.close()


with File("irrelevant.txt", "r") as f:
    10 / 0

__exit__ called


ZeroDivisionError: division by zero

### contextlib

In [134]:
from contextlib import contextmanager


@contextmanager
def File(name: str, mode: str = "r"):
    file_handle = None
    try:
        file_handle = open(name, mode)
        yield file_handle
    finally:
        if file_handle:
            print("closing")
            file_handle.close()

In [136]:
with File("irrelevant.txt", "r") as f:
    pass

closing


In [137]:
with File("3.txt", "r") as f:
    10 / 0

FileNotFoundError: [Errno 2] No such file or directory: '3.txt'

In [138]:
import sys
import datetime
from typing import Generator
from typing.io import TextIO
from contextlib import contextmanager



@contextmanager
def execution_time_printed(file: TextIO = sys.stdout) -> Generator[None, None, None]:
    start = datetime.datetime.now()
    yield
    print("Execution time:", datetime.datetime.now() - start, file=file)

In [139]:
with execution_time_printed():
    print("inside")
    import time
    time.sleep(0.5)
print("outside")

inside
Execution time: 0:00:00.500736
outside


### Reentrant contextmanagers

In [140]:
file = open("irrelevant.txt","w")
with file:
    file.write("a")


with file:
    file.write("a")

ValueError: I/O operation on closed file.

In [141]:
import sqlite3
db = sqlite3.connect(":memory:")
with db:
    db.execute("")
    db.execute("")

In [142]:
with db:
    db.execute("")
    db.execute("")

### Btw threading pool context manager

In [143]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(pow, 323, 1235)

print(future.result())

7330187419711662525292446729952277967765833457839424373869116780517420149076198183898894905236339668910883614705749593461204605714622607490668683222808958179218191780471063516227460897872227090014097232715359988867409006240081206335670819048146328123935337644631341482039026277834498173918554303355637321060412263715606736911839847081166018722332660742474936263046482602637679145832497919840537694829188335160914131310111239449199642739655793719812086149415859534959085359215402107080568853413877372159233452025442228651418507639010743174496936173262981681095359156359401217962539764203947129055258900852300663811552683018727645219707243611502505332407412509113706415784954450373499498470564461122438759199401776852200401606640428677779377601753155808428527723343610817938497649317098807976560094452142608380040084257453294862872175833371967399756248792099979580669182692895291737594003884626270278670234491588018888596571060169103937561057223083861024046092685665503687387204434227037124219374130642

In [142]:
from concurrent.futures import ProcessPoolExecutor, Executor

with ProcessPoolExecutor(max_workers=8) as executor:
    executor: Executor
    powers = list(executor.map(pow, range(100), range(100)))
powers

[1,
 1,
 4,
 27,
 256,
 3125,
 46656,
 823543,
 16777216,
 387420489,
 10000000000,
 285311670611,
 8916100448256,
 302875106592253,
 11112006825558016,
 437893890380859375,
 18446744073709551616,
 827240261886336764177,
 39346408075296537575424,
 1978419655660313589123979,
 104857600000000000000000000,
 5842587018385982521381124421,
 341427877364219557396646723584,
 20880467999847912034355032910567,
 1333735776850284124449081472843776,
 88817841970012523233890533447265625,
 6156119580207157310796674288400203776,
 443426488243037769948249630619149892803,
 33145523113253374862572728253364605812736,
 2567686153161211134561828214731016126483469,
 205891132094649000000000000000000000000000000,
 17069174130723235958610643029059314756044734431,
 1461501637330902918203684832716283019655932542976,
 129110040087761027839616029934664535539337183380513,
 11756638905368616011414050501310355554617941909569536,
 1102507499354148695951786433413508348166942596435546875,
 106387358923716524807713475752

### Other useful contextlib members

# TODO: multiple managers in one with

# TODO: 

https://docs.python.org/3/library/contextlib.html

https://www.rath.org/on-the-beauty-of-pythons-exitstack.html

### Multiple inheritance, method resolution order (mro)

In [ ]:
class A:
    def f(self):
        print("A")

class B:
    def f(self):
        print("B")

class C(B,A):
    pass

C().f()                             
print(C.mro())

In [ ]:
class C(A, B):
    pass

C().f()                             
print(C.mro())

In [50]:
class C(A, B):
    def f(self):
        B.f(self) # in general class.method(self) == object.method()

C().f()

NameError: name 'A' is not defined

### debugging - pdb

### Asyncio

In [ ]:
import asyncio
import time


async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    task1 = asyncio.create_task(say_after(1, "hello"))

    task2 = asyncio.create_task(say_after(2, "world"))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")

await main()

Normally you would use ```asyncio.run(main())```, jupyter (IPython) is already running an event loop

In [ ]:
import time
from queue import Queue
from threading import Thread


def producer_func(queue):
    print(f"Putting {1}")
    queue.put(1)
    print(f"Putting {2}")
    queue.put(2)
    print(f"Producer waiting for more tasks")
    time.sleep(2)
    print(f"Putting {3}")
    queue.put(3)
    print(f"Producer shuting down")
    queue.put(None)


def consumer_func(queue):
    while True:
        task = queue.get()
        if task is None:
            print(f"Got None, exiting")
            queue.task_done()
            break
        time.sleep(0.5)
        queue.task_done()
        print(f"Task Done {task}")

queue = Queue()
producer = Thread(target=producer_func(queue))
consumer = Thread(target=consumer_func(queue))
producer.start()
consumer.start()
queue.join()

In [ ]:
import asyncio
import time


async def producer(queue):
    print(f"Putting {1}")
    await queue.put(1)
    print(f"Putting {2}")
    await queue.put(2)
    print(f"Producer waiting for more tasks")
    await asyncio.sleep(2)
    print(f"Putting {3}")
    await queue.put(3)
    print(f"Producer shuting down")
    await queue.put(None)


async def consumer(queue):
    while True:
        task = await queue.get()
        if task is None:
            print(f"Got None, exiting")
            queue.task_done()
            break
        time.sleep(0.5)
        queue.task_done()
        print(f"Task Done {task}")


async def main():
    queue = asyncio.queues.Queue()
    producer_coro = asyncio.create_task(producer(queue))
    consumer_coro = asyncio.create_task(consumer(queue))
    await producer_coro
    await consumer_coro
    await queue.join()


await main()


### Itertools

In [ ]:
import itertools

In [ ]:
list(itertools.chain([1, 2], (3, 4, 5), "6"))

In [ ]:
list(itertools.repeat(1,5))

In [ ]:
list(itertools.islice(itertools.count(), 4))

### Enumerate

In [ ]:
values = ["a","b"]

In [ ]:
# DONT DO THIS:
for i in range(len(values)):
    print(f"values[{i}] = {values[i]}")

In [ ]:
for index, value in enumerate(values):
    print(f"values[{index}] = {value}")

In [ ]:
list(enumerate(values)) == [(0, "a"), (1, "b")]

# TODO: dynamic class creation

# TODO: virtualenv, python -m

# TODO: Imports :(

# TODO: dynamic class creation

# TODO: microbenchmarking

### super() TODO

In [ ]:
class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)

class Cube(Square):
    def surface_area(self):
        face_area = super().area()
        return face_area * 6

    def volume(self):
        face_area = super().area()
        return face_area * self.length